In [1]:
import harmonize as hz
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
from shapely.geometry import mapping

In [17]:
path_data = "../data/"

In [21]:
old_evap = xr.open_dataset(path_data +"Raw/Raw_Evap_8D_500m.nc")

In [22]:
old_evap

<xarray.Dataset>
Dimensions:     (time: 584, ydim: 505, xdim: 595)
Coordinates:
  * time        (time) object 2009-12-27 00:00:00 ... 2022-08-29 00:00:00
  * ydim        (ydim) float64 5.018e+06 5.017e+06 ... 4.785e+06 4.784e+06
  * xdim        (xdim) float64 3.412e+05 3.417e+05 ... 6.16e+05 6.164e+05
Data variables:
    crs         int8 ...
    ET_500m     (time, ydim, xdim) float32 ...
    ET_QC_500m  (time, ydim, xdim) float32 ...
    LE_500m     (time, ydim, xdim) float32 ...
    PET_500m    (time, ydim, xdim) float32 ...
    PLE_500m    (time, ydim, xdim) float32 ...
Attributes:
    title:        MOD16A2.006 for aid0001
    Conventions:  CF-1.6
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.12
    references:   See README.md
    history:      See README.md

In [27]:
datacube2 = xr.open_dataset(path_data +"final/datacube2.nc")

In [28]:
datacube2

<xarray.Dataset>
Dimensions:            (x: 248, y: 217, time: 366)
Coordinates:
  * x                  (x) float64 3.294e+05 3.303e+05 ... 5.574e+05 5.583e+05
  * y                  (y) float64 5.032e+06 5.031e+06 ... 4.833e+06 4.832e+06
  * time               (time) object 2010-01-01 00:00:00 ... 2011-01-01 00:00:00
    band               int32 ...
Data variables: (12/15)
    crs                int32 ...
    Fpar_500m          (time, y, x) float64 ...
    spatial_ref        int32 ...
    density            (y, x) float32 ...
    u10                (time, y, x) float64 ...
    v10                (time, y, x) float64 ...
    ...                 ...
    Last_Day           (time, y, x) float64 ...
    Burn_Date          (time, y, x) float64 ...
    FireMask           (time, y, x) float32 ...
    LST_Day_1km        (time, y, x) float64 ...
    LST_Night_1km      (time, y, x) float64 ...
    _1_km_16_days_EVI  (time, y, x) float64 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [20]:
evap = xr.open_dataset(path_data + 'final/final_evap_8D_500m.nc')

In [5]:
evap_filter = evap['ET_500m']

In [6]:
evap_filter = evap_filter.sel(time=slice('2010-01-01', '2011-01-01'))

In [7]:
crs_sinu = rasterio.crs.CRS.from_string(
        "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs")

In [8]:
evap_filter = hz.define_crs(evap_filter, crs_sinu)

In [9]:
aoi = hz.define_area_of_interest(path_data + 'Large.zip')

In [10]:
common_grid = rxr.open_rasterio(path_data + 'final_lst_day_1D_1km.nc').isel(time=0)

In [14]:
evap

<xarray.Dataset>
Dimensions:     (time: 77, ydim: 434, xdim: 494)
Coordinates:
  * time        (time) object 2021-01-01 00:00:00 ... 2022-08-29 00:00:00
  * ydim        (ydim) float64 5.032e+06 5.032e+06 ... 4.832e+06 4.832e+06
  * xdim        (xdim) float64 3.296e+05 3.301e+05 ... 5.576e+05 5.581e+05
Data variables:
    crs         int8 ...
    ET_500m     (time, ydim, xdim) float32 ...
    ET_QC_500m  (time, ydim, xdim) float32 ...
    LE_500m     (time, ydim, xdim) float32 ...
    PET_500m    (time, ydim, xdim) float32 ...
    PLE_500m    (time, ydim, xdim) float32 ...
Attributes:
    title:        MOD16A2.061 for aid0001
    Conventions:  CF-1.6
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.14
    references:   See README.md
    history:      See README.md

In [12]:
# interpolate to common grid
evap_filter = hz.interpolate_to_common_grid(evap_filter, common_grid)

CPLE_IllegalArgError: GDALWarpOptions.Validate(): nBandCount=0, no bands configured!